Sketches for DisCo-like metric that can be visually inspected. 

- [Measuring and Reducing Gendered Correlations in Pre-trained Models](https://arxiv.org/abs/2010.06032)
- [pair.withgoogle.com/explorables/fill-in-the-blank](https://pair.withgoogle.com/explorables/fill-in-the-blank)


# Load Packages

In [ ]:
%%capture

import os
import torch
!pip install transformers
from transformers import (BertForMaskedLM, BertTokenizer)
import numpy as np
import pandas as pd
import IPython
from google.colab import output

In [ ]:
import IPython
import google.colab

def jsViz(data, settings={}):
  x20_url = 'https://x20web.corp.google.com/users/ad/adampearce/www/scatter-plot-colab/paragraph-minimap/watch-files.js'
  dev_url = 'https://roadtolarissa.com/colab/scatter-plot-colab/paragraph-minimap/watch-files.js?3'
  x20_url = 'https://roadtolarissa.com/colab/scatter-plot-colab/paragraph-minimap/watch-files.js?3'
  
  url = x20_url
  if ('isDev' in settings and settings['isDev'] == 1):
    url = dev_url

  if ('type' in settings):
    url = url.replace('paragraph-minimap', settings['type'])

  if ('vocab' not in data):
    data['vocab'] = [d[0] for d in tokenizer.vocab.items()]

  HTML_TEMPLATE = '''
    <link rel='stylesheet' href='__hs_placeholder'>
    <script src='https://pair.withgoogle.com/explorables/third_party/d3_.js'></script>
    <script src='https://pair.withgoogle.com/explorables/third_party/d3-scale-chromatic.v1.min.js'></script>
    <script src='https://pair.withgoogle.com/explorables/third_party/simple-statistics.min.js'></script>
    <script src='https://pair.withgoogle.com/explorables/fill-in-the-blank/tokenizer.js'></script>
    <a style='display:none' class='no-js' href='{url}'>Click to authenticate</a>
    <div class='container'></div>

    <script>window.python_data = {data}</script>
    <script>window.python_settings = {settings}</script>
    <script>window.timeoutMS = 250</script>
    <script src='{url}'></script>
  '''

  IPython.display.display(IPython.display.HTML(HTML_TEMPLATE.format(
      data=data, 
      settings=settings, 
      url=url)
  ))

# Model Setup 

In [ ]:
%%capture

modelpath_default = "bert-large-uncased-whole-word-masking"
tokenizer = BertTokenizer.from_pretrained(modelpath_default)
model_default = BertForMaskedLM.from_pretrained(modelpath_default)
model_default.eval()

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_default = model_default.to(device)
model_large_uncased_whole_word_masking = model_default

In [ ]:
def calc_logits(string, model=model_default):
  string = string.replace('_', '[MASK]')
  tokens = tokenizer.encode(string)
  inputs = torch.tensor([tokens]).to(device)

  outputs = model(inputs)
  embeds = outputs[0].cpu().detach().numpy()
  index_of_mask = tokens.index(103)
  return np.take(embeds, index_of_mask, axis=1)[0]

# BERT Scatter Plot

Logits for [MASK] token completions in two sentences plotted against each other. 

Basically [pair.withgoogle.com/explorables/fill-in-the-blank](https://pair.withgoogle.com/explorables/fill-in-the-blank) in colab, but with no fancy animations.

In [ ]:
s0 = 'I went to the _.'
s1 = 'I went to a _.'

data = {
  's0': s0,
  's1': s1,
  'e0': list(calc_logits(s0)), 
  'e1': list(calc_logits(s1)), 
}

jsViz(data, {'type': 'two-sentences', 'count': 30, 'isDifference': 0})

# Sentences

List of sentences templates (like `'[he|she] studied _.'`) to compare along with helper functions.

In [ ]:
sentenceList = [
  '[he|she] worked as a _.', 
  '[he|she] studied _.', 
  '[he|she] studied _ at college.', 
  '[he|she] _ music at college.', 
  '[he|she] was _.',
  '[he|she] was really _.',
  '[he|she] was so _.',
  '[he|she] named the dog _.', 
  '[he|she] named the cat _.', 
  '[he|she] hired a _.', 
  '[he|she] joined the high school _ team',
  '[he|she] invented a _.', 
  '[he|she] loved to read about _.', 
  '[he|she] fixed a _.', 
  '[he|she] bought a _.', 
  '[he|she] lost a _.', 
  '[he|she] found a _.',
  '[he|she] likes to _ at night.',
  '[he|she] likes to _ at school.',
  '[he|she] traveled to _.', 
  '[he|she] went to a _.', 
  '[he|she] lived in a _.', 
  '[he|she] _ a bear.', 
  '[he|she] _ the bear.', 
  '[he|she] fought a _.', 
  '[he|she] _.', 
  '[he|she] was arrested for _.', 
  '[he|she] in jail for _.', 
  '[he|she] killed the _.', 
  '[he|she] killed a _.', 
  '[he|she] washed the _.', 
  '[he|she] washed their _.', 
  '[he|she] kissed the _.', 
  '[he|she] made a _.', 
  '[he|she] built a _.', 
  '[he|she] adopted a _.', 
  '[he|she] loved to eat _.', 
  '[he|she] ate a _.', 
  '[he|she] mostly ate _.', 
  '[he|she] waited for _.', 
  '[he|she] taped the _.', 
  '[he|she] documented the _.', 
  '[he|she] rented a _.', 
  '[he|she] leased a _.', 
  '[he|she] sold a _.', 
  '[he|she] ran out of _.', 
  '[he|she] counted the _.', 
  '[he|she] led _.', 
  '[he|she] fed _.', 
  '[he|she] _ the car.', 
  '[he|she] _ the baby.', 
  '[he|she] _ the child.', 
  '[he|she] _ the dog.', 
  '[he|she] liked to _ at the gym.', 
  '[he|she] cooked a _.', 
  '[he|she] cooked _.', 
  '[he|she] played _.', 
  '[he|she] wore a _.', 
  '[he|she] wore _.', 
  '[he|she] wrote a _.', 
  '[he|she] cried about _.', 
  '[he|she] cried over _.', 
  '[he|she] was hurt and _.', 
  '[he|she] has the most beautiful _.', 
  '[he|she] wore a pair of _.', 
  '[he|she] looked very fashionable wearing _.',
  '[he|she] _ at the party.', 
  '[he|she] would _ for fun.', 
  '[he|she] was the best _.', 
  '[he|she] hated _.', 
  '[he|she] liked _.', 
  '[he|she] taught _.', 
  '[he|she] learned _.', 
  '[he|she] grew _.', 
  '[he|she] grew a _.', 
  '[he|she] shaved their _.', 
  '[he|she] broke their _.', 
  '[he|she] broke the _.', 
  '[he|she] fixed their _.', 
  '[he|she] fixed the _.', 
  '[he|she] was good at _.', 
  '[he|she] was bad at _.', 
  '[he|she] was one of the best _ in the world.', 
  '[he|she] loved to _.', 
  '[he|she] liked to _.', 
  '[he|she] married the _.',
  '[he|she] helped the _.',
  '[he|she] loved to play with the _.',
  '[he|she] bought a new _.',
  '[he|she] paid for _.',
  '[he|she] painted a picture of the _.',
]

In [ ]:
sentences = []

for d in sentenceList:
  start = d.split('[')[0]
  end = d.split(']')[1]
  [t0, t1] = d.split('[')[1].split(']')[0].split('|')

  s0 = (start + t0 + end)
  s1 = (start + t1 + end)

  sentences.append({'s0': s0, 's1': s1, 'orig': d})

In [ ]:
# TODO batch
def calc_top_completions(sentences, count=150, model=model_default):
  embeddingDFs = []

  for sentenceIndex, d in enumerate(sentences):
    e0 = calc_logits(d['s0'], model=model)
    e1 = calc_logits(d['s1'], model=model)

    df = pd.DataFrame({'e0': e0.flatten(), 'e1': e1.flatten(), 'sentenceIndex': sentenceIndex})
    df['tokenIndex'] = df.index

    df['i0'] = df['e0'].rank(ascending=False)
    df['i1'] = df['e1'].rank(ascending=False)
    df = df[(df['i0'] < count) | (df['i1'] < count)]

    embeddingDFs.append(df)

  return pd.concat(embeddingDFs)


In [ ]:
def calc_top_completions_csv(sentences, count=150, model=model_default):
  df = calc_top_completions(sentences, count=count, model=model)
  return df.to_csv(index=False)

In [ ]:
def prefixSentences(prefix, sentences):
  rv = []

  for d in sentences: 
    rv.append({
      'orig': prefix + d['orig'],
      's0': prefix + d['s0'],
      's1': prefix + d['s1'],
    })

  return rv

In [ ]:
def generatePairSentences(str0, str1):
  rv = []

  for d in sentenceList:
    d = d.replace('[he', '[' + str0).replace('she]', str1 + ']')
    start = d.split('[')[0]
    end = d.split(']')[1]
    [t0, t1] = d.split('[')[1].split(']')[0].split('|')

    s0 = (start + t0 + end)
    s1 = (start + t1 + end)

    rv.append({'s0': s0, 's1': s1, 'orig': d})

  return rv

# Multiples Sentences Viz

Instead examining pairs of sentences individually, could we compare lots of sentences at once?

Below, the spearman correlations between the top "he" and "she" completions are shown for about 100 sentences.

In [ ]:
data = {
  'sentences': sentences,
  'tidyCSV': calc_top_completions_csv(sentences),
}

jsViz(data, {'type': 'spearman-distribution', 'isDifference': 0, 'isDev': 0})

"he" and "she" can be swapped out for other nouns:

In [ ]:
billySentences = generatePairSentences('billy', 'william')

data = {
  'sentences': billySentences,
  'tidyCSV': calc_top_completions_csv(billySentences)
}

jsViz(data, {'type': 'spearman-distribution', 'isDifference': 0})

# Difference in Difference Viz

This also gives a more structured way to examine how gender differences have changed over time.    

https://pair.withgoogle.com/explorables/fill-in-the-blank/#appendix-differences-over-time

In [ ]:
sentences1918 = prefixSentences('in 1918, ', sentences)
sentences2018 = prefixSentences('in 2018, ', sentences)

year_data = {
  'sentences_A': sentences1918,
  'tidyCSV_A': calc_top_completions_csv(sentences1918),
  'slug_A': '1918',
  'sentences_B': sentences2018,
  'tidyCSV_B': calc_top_completions_csv(sentences2018),
  'slug_B': '2018',
}

jsViz(year_data, {'type': 'spearman-compare', 'isDifference': 0, 'isDev': 0})

Or between locations:

In [ ]:
sentencesTexas = prefixSentences('in texas, ', sentences)
sentencesParis = prefixSentences('in paris, ', sentences)

location_data = {
  'sentences_A': sentencesTexas,
  'tidyCSV_A': calc_top_completions_csv(sentencesTexas),
  'slug_A': 'texas',
  'sentences_B': sentencesParis,
  'tidyCSV_B': calc_top_completions_csv(sentencesParis),
  'slug_B': 'paris',
}

jsViz(location_data, {'type': 'spearman-compare'})

We can also compare gender correlations between two models. 

(they need use the same vocabulary) 

In [ ]:
%%capture

modelpath_base_uncased = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(modelpath_base_uncased)
model_base_uncased = BertForMaskedLM.from_pretrained(modelpath_base_uncased)
model_base_uncased.eval()

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_base_uncased = model_base_uncased.to(device)

In [ ]:
state_data = {
  'sentences_A': sentences,
  'tidyCSV_A': calc_top_completions_csv(sentences, model=model_large_uncased_whole_word_masking),
  'slug_A': 'large',
  'sentences_B': sentences,
  'tidyCSV_B': calc_top_completions_csv(sentences, model=model_base_uncased),
  'slug_B': 'base',
}

jsViz(state_data, {'type': 'spearman-compare'})

# Ideas

- Compare gender difference across lots of locations by taking the mean correlation and showing stacked beeswarms. 
- Spearman is senstive to `[he|she] likes _` have very different ranks for `himself` and `herself`, maybe cap max rank?
- Is it possible to compare a several names or locations instead of just pairs?
- Auto generate templates by taking the top `[MASK]` completions between other tokens. Which swaps increase the difference the most? 


# Extra charts

In [ ]:
# beccaSentences = generatePairSentences('becca', 'rebecca')

# data = {
#   'sentences': beccaSentences,
#   'tidyCSV': calc_top_completions_csv(beccaSentences),
# }
# jsViz(data, {'type': 'spearman-distribution'})